In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import json, joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Pretty numbers in notebook (no scientific notation)
pd.set_option('display.float_format', lambda x: f'{x:,.4f}')
np.set_printoptions(suppress=True)

# Paths
DATA_CSV = "clean.csv"
OUT_DIR = Path("out"); OUT_DIR.mkdir(exist_ok=True)
RANDOM_STATE = 42

In [2]:
# Load
df = pd.read_csv(DATA_CSV, encoding="utf-8-sig")

# Expected Arabic columns (from your screenshot)
COL_SECTOR   = "القطاع"
COL_SYMBOL   = "رمز الشركة"
COL_NAME     = "اسم الشركة"
COL_DATE     = "التاريخ"
COL_OPEN     = "إفتتاح"
COL_HIGH     = "الأعلى"
COL_LOW      = "الأدنى"
COL_CLOSE    = "إقفال"
COL_CHANGE   = "التغيير"
COL_PCT      = "% التغيير"
COL_VOLUME   = "الكمية المتداولة"
COL_TURNOVER = "اجمالي القيمة المتداولة (ر.س)"
COL_TRADES   = "عدد الصفقات"

# Parse date
df[COL_DATE] = pd.to_datetime(df[COL_DATE], errors="coerce")

# Ensure numeric for features
num_cols = [COL_OPEN, COL_HIGH, COL_LOW, COL_CLOSE, COL_CHANGE, COL_PCT,
            COL_VOLUME, COL_TURNOVER, COL_TRADES]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Drop rows with missing key fields
df = df.dropna(subset=[COL_DATE, COL_SYMBOL, COL_CLOSE]).copy()

print(df.shape)
df.head()

(249300, 13)


,القطاع,رمز الشركة,اسم الشركة,التاريخ,إفتتاح,الأعلى,الأدنى,إقفال,التغيير,% التغيير,الكمية المتداولة,اجمالي القيمة المتداولة (ر.س),عدد الصفقات
0,البنوك,1010,بنك الرياض,2025-05-28,28.0000,28.5000,27.8500,28.1000,0.1000,0.3600,3544508,"99,916,754.1500",3302
1,البنوك,1010,بنك الرياض,2025-05-29,28.2500,29.0500,28.0000,29.0500,0.9500,3.3800,6007732,"172,610,047.4000",3181
2,البنوك,1010,بنك الرياض,2025-05-27,28.0000,28.0500,27.5500,28.0000,0.0000,0.0000,1928922,"53,653,332.8500",2106
3,البنوك,1010,بنك الرياض,2025-05-25,28.3500,28.3500,28.1000,28.2500,-0.1000,-0.3500,529542,"14,941,194.8000",808
4,البنوك,1010,بنك الرياض,2025-05-26,28.6500,28.6500,27.8000,28.0000,-0.2500,-0.8800,1763003,"49,366,287.0000",2445


In [8]:
# ---- NEXT-DAY targets (no label leakage) ----
df = df.sort_values([COL_SYMBOL, COL_DATE]).copy()

# Next day's close & % change per symbol
df["next_close"] = df.groupby(COL_SYMBOL)[COL_CLOSE].shift(-1)
df["next_pct"]   = df.groupby(COL_SYMBOL)[COL_PCT].shift(-1)

# A) change-based NEXT DAY: BUY if next day's % change > 0
df["target_change_next"] = (df["next_pct"] > 0).astype(float)

# B) close-based NEXT DAY: BUY if next day's close > today's close
df["target_close_next"]  = (df["next_close"] > df[COL_CLOSE]).astype(float)

# drop rows where next day doesn't exist (last row per symbol)
df = df.dropna(subset=["target_change_next", "target_close_next"]).copy()
df["target_change_next"] = df["target_change_next"].astype(int)
df["target_close_next"]  = df["target_close_next"].astype(int)

print(
    df[[COL_SYMBOL, COL_DATE, COL_CLOSE, "next_close", COL_PCT, "next_pct",
        "target_change_next","target_close_next"]].head(10)
)

      رمز الشركة    التاريخ   إقفال  next_close  % التغيير  next_pct  \
2369        1010 2015-08-25 14.9900     14.8100     8.0700   -1.2000   
2370        1010 2015-08-26 14.8100     14.8500    -1.2000    0.2700   
2371        1010 2015-08-27 14.8500     14.8700     0.2700    0.1300   
2372        1010 2015-08-30 14.8700     14.5400     0.1300   -2.2200   
2373        1010 2015-08-31 14.5400     14.5800    -2.2200    0.2800   
2374        1010 2015-09-01 14.5800     14.4000     0.2800   -1.2300   
2375        1010 2015-09-02 14.4000     14.4300    -1.2300    0.2100   
2376        1010 2015-09-03 14.4300     14.4400     0.2100    0.0700   
2377        1010 2015-09-06 14.4400     14.3400     0.0700   -0.6900   
2378        1010 2015-09-07 14.3400     14.5000    -0.6900    1.1200   

      target_change_next  target_close_next  
2369                   0                  0  
2370                   1                  1  
2371                   1                  1  
2372                   

In [14]:
# ========= Cell 4 (REPLACE THIS CELL) =========
# Feature set WITHOUT 'عدد الصفقات (trades)'
FEATURES = [
    COL_OPEN, COL_HIGH, COL_LOW, COL_CLOSE, COL_CHANGE, COL_PCT,
    COL_VOLUME, COL_TURNOVER
]
FEATURES = [c for c in FEATURES if c in df.columns]  # keep only existing

print("Using features (NO trades):", FEATURES)

def train_and_eval_model(X, y, name, out_prefix):
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import accuracy_score, classification_report
    import joblib, json

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
    )

    clf = RandomForestClassifier(
        n_estimators=300,
        min_samples_leaf=2,
        random_state=RANDOM_STATE,
        n_jobs=-1
    )
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    rep = classification_report(y_test, y_pred, output_dict=True, zero_division=0)

    # NOTE: new filenames so we don't confuse with the old (with-trades) models
    model_path = OUT_DIR / f"{out_prefix}_model_notrades.joblib"
    joblib.dump(clf, model_path)
    with open(OUT_DIR / f"{out_prefix}_metrics_notrades.json", "w", encoding="utf-8") as f:
        json.dump({"accuracy": acc, "report": rep}, f, ensure_ascii=False, indent=2)

    print(f"{name} accuracy (NO trades): {acc:.4f}")
    return clf, acc

Using features (NO trades): ['إفتتاح', 'الأعلى', 'الأدنى', 'إقفال', 'التغيير', '% التغيير', 'الكمية المتداولة', 'اجمالي القيمة المتداولة (ر.س)']


In [15]:
# ========= Cell 5 (REPLACE THIS CELL — time-based split, NO trades) =========

# Time-based split: last 20% (by date) per symbol = test
def time_split_per_symbol(frame, test_frac=0.20):
    parts = []
    for sym, g in frame.sort_values(COL_DATE).groupby(COL_SYMBOL):
        n = len(g)
        k = max(1, int(n * (1 - test_frac)))
        g = g.copy()
        g["is_test"] = False
        g.iloc[k:, g.columns.get_loc("is_test")] = True
        parts.append(g)
    return pd.concat(parts, ignore_index=True)

# Prepare two datasets sharing same FEATURES (NO trades)
use_cols = FEATURES + ["target_change_next","target_close_next", COL_SYMBOL, COL_DATE]
work = df.dropna(subset=FEATURES).copy()[use_cols]

# Split
split = time_split_per_symbol(work, test_frac=0.20)

def train_eval_time(df_split, y_col, name, out_prefix):
    train = df_split[df_split["is_test"] == False]
    test  = df_split[df_split["is_test"] == True]

    X_train = train[FEATURES].astype(np.float64).values
    y_train = train[y_col].values
    X_test  = test[FEATURES].astype(np.float64).values
    y_test  = test[y_col].values

    clf = RandomForestClassifier(
        n_estimators=300,
        min_samples_leaf=2,
        random_state=RANDOM_STATE,
        n_jobs=-1
    )
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    rep = classification_report(y_test, y_pred, output_dict=True, zero_division=0)

    # Save model + metrics with _notrades suffix
    model_path = OUT_DIR / f"{out_prefix}_model_notrades.joblib"
    joblib.dump(clf, model_path)
    with open(OUT_DIR / f"{out_prefix}_metrics_notrades.json", "w", encoding="utf-8") as f:
        json.dump({"accuracy": float(acc), "report": rep}, f, ensure_ascii=False, indent=2)

    print(f"{name} (next-day, NO trades) accuracy: {acc:.4f}")
    return clf, acc

# A) change-based (next day)
clf_change, acc_change = train_eval_time(split, "target_change_next", "Change-based", "change")

# B) close-based (next day)
clf_close, acc_close   = train_eval_time(split, "target_close_next",  "Close-based",  "close")

# Pick better
if acc_close >= acc_change:
    best_name = "close_notrades"
    best_clf  = clf_close
    best_acc  = acc_close
    best_target = "target_close_next"
else:
    best_name = "change_notrades"
    best_clf  = clf_change
    best_acc  = acc_change
    best_target = "target_change_next"

print(f"\n✅ Best model (NO trades): {best_name} (accuracy={best_acc:.4f})")

Change-based (next-day, NO trades) accuracy: 0.5217
Close-based (next-day, NO trades) accuracy: 0.5263

✅ Best model (NO trades): close_notrades (accuracy=0.5263)


In [16]:
# ===================== Cell 6 — SCORE & KEEP ALL ROWS =====================

# 1) sort & keep rows that have all features
scored_all = df.sort_values([COL_SYMBOL, COL_DATE]).copy()
scored_all = scored_all.dropna(subset=FEATURES).copy()

# 2) model probabilities (always pick the column for class==1 i.e., BUY)
X_all = scored_all[FEATURES].astype(np.float64).values
proba_all = best_clf.predict_proba(X_all)
buy_col = np.where(best_clf.classes_ == 1)[0][0]
scored_all["proba_buy"] = proba_all[:, buy_col]
scored_all["decision"]  = np.where(scored_all["proba_buy"] >= 0.5, "اشتري", "لا تشتري")

# 3) align each row's prediction to the NEXT day (T -> T+1)
scored_all["pred_date"] = scored_all.groupby(COL_SYMBOL)[COL_DATE].shift(-1)

# 4) for the LAST day per symbol (no T+1): keep the row and INHERIT previous class
mask_last = scored_all["pred_date"].isna()

# pred_date becomes the same day's date (so we still output something for that date)
scored_all.loc[mask_last, "pred_date"] = scored_all.loc[mask_last, COL_DATE]

# inherit yesterday's predicted decision/proba (the signal meant for "today")
prev_decision = scored_all.groupby(COL_SYMBOL)["decision"].shift(1)
prev_proba    = scored_all.groupby(COL_SYMBOL)["proba_buy"].shift(1)
scored_all.loc[mask_last, "decision"]  = prev_decision[mask_last]
scored_all.loc[mask_last, "proba_buy"] = prev_proba[mask_last]

# if the very first row of a symbol had no previous signal, fill a safe default
scored_all["decision"].fillna("لا تشتري", inplace=True)
scored_all["proba_buy"].fillna(0.0, inplace=True)

# (optional) keep the feature day for clarity
scored_all["today_date"] = scored_all[COL_DATE]

# 5) export: one row per (symbol, date) where "date" is the day the decision applies to
export_cols = [COL_SYMBOL, COL_NAME, "today_date", "pred_date", COL_CLOSE, COL_PCT, "proba_buy", "decision"]
scored_all_out = scored_all[export_cols].rename(columns={
    COL_SYMBOL: "symbol",
    COL_NAME:   "company",
    "pred_date":"date",      # the day the decision applies to
    COL_CLOSE:  "close",
    COL_PCT:    "pct_change"
})

out_path = OUT_DIR / "predictions_all_dates_next.csv"
scored_all_out.to_csv(out_path, index=False, encoding="utf-8-sig")
print("✅ wrote:", out_path)

# sanity: lengths should match (no rows dropped because of last day)
print("rows in clean (with features):", len(df.dropna(subset=FEATURES)))
print("rows in predictions file     :", len(scored_all_out))

# sanity: no 'اشتري' with proba<0.5 (after inheritance)
bad = (scored_all_out["decision"] == "اشتري") & (scored_all_out["proba_buy"] < 0.5)
print("rows with اشتري but proba<0.5:", int(bad.sum()))

C:\Users\asma5\AppData\Local\Temp\ipykernel_22268\1418640574.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  scored_all["decision"].fillna("لا تشتري", inplace=True)
C:\Users\asma5\AppData\Local\Temp\ipykernel_22268\1418640574.py:31: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

✅ wrote: out\predictions_all_dates_next.csv
rows in clean (with features): 249300
rows in predictions file     : 249300
rows with اشتري but proba<0.5: 0


In [18]:
# ===================== Cell 7 — Batch score Firestore (NO trades) =====================
# Reads ALL days in companies/*/PriceRecords_full, predicts T->T+1, keeps last day, writes to market_predictions_daily

import pandas as pd, numpy as np, datetime as dt, re
from pathlib import Path
import joblib

import firebase_admin
from firebase_admin import credentials, firestore

# --- CONFIG (edit paths if needed) ---
SERVICE_ACCOUNT_JSON = r"nomu-47a92-firebase-adminsdk-fbsvc-1b2e28026c.json"       # <-- change me
MODEL_PATH           = Path("out/close_model_notrades.joblib")    # or out/change_model_notrades.joblib
COMPANIES_COLLECTION = "companies"
PRICE_SUBCOLLECTION  = "PriceRecords_full"
TARGET_DAILY_COL     = "market_predictions_daily"

# Arabic labels used in your training notebook
COL_SYMBOL   = "رمز الشركة"
COL_NAME     = "اسم الشركة"
COL_DATE     = "التاريخ"
COL_OPEN     = "إفتتاح"
COL_HIGH     = "الأعلى"
COL_LOW      = "الأدنى"
COL_CLOSE    = "إقفال"
COL_CHANGE   = "التغيير"
COL_PCT      = "% التغيير"
COL_VOLUME   = "الكمية المتداولة"
COL_TURNOVER = "اجمالي القيمة المتداولة (ر.س)"

# 8 features (NO trades)
FEATURES = [COL_OPEN, COL_HIGH, COL_LOW, COL_CLOSE, COL_CHANGE, COL_PCT, COL_VOLUME, COL_TURNOVER]

# Map Firestore field names -> your model column names
FS_TO_MODEL = {
    "open":        COL_OPEN,
    "high":        COL_HIGH,
    "low":         COL_LOW,
    "close":       COL_CLOSE,
    "change":      COL_CHANGE,
    "change_pct":  COL_PCT,
    "volume":      COL_VOLUME,
    "value":       COL_TURNOVER,   # Firestore uses 'value' for turnover
    "date":        COL_DATE,
}

def norm_key(s: str) -> str:
    s = str(s)
    s = s.replace("إ","ا").replace("أ","ا").replace("آ","ا").replace("ٱ","ا").replace("ى","ي").replace("ة","ه")
    return re.sub(r"\s+", "", s).lower()

def pick(d: dict, *cands, default=None):
    """Pick first existing key among candidates (arabic/normalized)."""
    if not d: return default
    # direct
    for k in cands:
        if k in d and d[k] is not None:
            return d[k]
    # normalized
    nd = {norm_key(k): v for k, v in d.items()}
    for k in cands:
        nk = norm_key(k)
        if nk in nd and nd[nk] is not None:
            return nd[nk]
    return default

# --- Firestore init + model load ---
if not firebase_admin._apps:
    cred = credentials.Certificate(SERVICE_ACCOUNT_JSON)
    firebase_admin.initialize_app(cred)
db = firestore.client()

clf = joblib.load(MODEL_PATH)
print("✅ Loaded model:", MODEL_PATH)

# --- Pull ALL companies and ALL their daily records ---
rows = []
companies = list(db.collection(COMPANIES_COLLECTION).stream())
print("Companies found:", len(companies))

for comp in companies:
    c = comp.to_dict() or {}
    symbol = str(pick(c, "id", "symbol", COL_SYMBOL, default=comp.id)).strip()
    name   = pick(c, "name", COL_NAME, default=comp.id)

    price_ref = db.collection(COMPANIES_COLLECTION).document(comp.id).collection(PRICE_SUBCOLLECTION)
    for rec in price_ref.stream():  # streams ALL docs in that subcollection
        d = rec.to_dict() or {}
        row = {"symbol": symbol, "company": name}
        for fs_key, model_key in FS_TO_MODEL.items():
            row[model_key] = pick(d, fs_key, model_key)
        rows.append(row)

df = pd.DataFrame(rows)
if df.empty:
    raise RuntimeError("No data found under companies/*/PriceRecords_full")
print("Pulled rows:", len(df))

# --- Types & cleaning ---
df[COL_DATE] = pd.to_datetime(df[COL_DATE], errors="coerce")
for c in FEATURES:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# Keep essential rows
df = df.dropna(subset=[COL_DATE, COL_CLOSE]).copy()
df["symbol"] = df["symbol"].astype(str).str.strip()

# If any feature column is entirely missing, create it with safe default
fill_defaults = {
    COL_OPEN: np.nan, COL_HIGH: np.nan, COL_LOW: np.nan, COL_CLOSE: np.nan,
    COL_CHANGE: 0.0, COL_PCT: 0.0, COL_VOLUME: 0.0, COL_TURNOVER: 0.0
}
for f in FEATURES:
    if f not in df.columns:
        df[f] = fill_defaults.get(f, 0.0)

# --- Predict BUY probability for ALL DAYS ---
df = df.sort_values(["symbol", COL_DATE]).copy()

# build a per-column defaults dict, then fill and cast
fill_map = {c: fill_defaults.get(c, 0.0) for c in FEATURES}
X_df = df[FEATURES].fillna(fill_map).astype(np.float64)

X = X_df.values
proba = clf.predict_proba(X)
buy_col = np.where(clf.classes_ == 1)[0][0]    # robustly choose BUY column
df["proba_buy"] = proba[:, buy_col]
df["decision"]  = np.where(df["proba_buy"] >= 0.5, "اشتري", "لا تشتري")

# --- Align prediction to NEXT day and KEEP last day via inheritance ---
df["pred_date"] = df.groupby("symbol")[COL_DATE].shift(-1)

mask_last = df["pred_date"].isna()
df.loc[mask_last, "pred_date"] = df.loc[mask_last, COL_DATE]          # keep last day
df["decision"]  = df.groupby("symbol")["decision"].ffill()
df["proba_buy"] = df.groupby("symbol")["proba_buy"].ffill()
df["decision"].fillna("لا تشتري", inplace=True)
df["proba_buy"].fillna(0.0, inplace=True)

# --- Prepare rows to write (with company, change, pct_change, classification, OHLC) ---
to_write = df[
    [
        "symbol", "company", "pred_date",
        COL_OPEN, COL_HIGH, COL_LOW, COL_CLOSE,
        COL_CHANGE, COL_PCT,
        "proba_buy", "decision"
    ]
].copy()

to_write = to_write.rename(columns={
    "pred_date": "date",
    COL_OPEN: "open",
    COL_HIGH: "high",
    COL_LOW:  "low",
    COL_CLOSE:"close",
    COL_CHANGE:"change",
    COL_PCT:  "pct_change",
})

# --- Write to Firestore: market_predictions_daily/{symbol_YYYY-MM-DD} ---
def make_doc_id(symbol, date_str):
    return f"{symbol}_{pd.to_datetime(date_str).date().isoformat()}"

now_iso = dt.datetime.utcnow().isoformat() + "Z"
batch = db.batch()
BATCH_LIMIT = 500
ops = 0
written = 0

for _, r in to_write.iterrows():
    sid = str(r["symbol"]).strip()
    d   = pd.to_datetime(r["date"]).date().isoformat()
    ref = db.collection(TARGET_DAILY_COL).document(make_doc_id(sid, d))

    payload = {
        "symbol": sid,
        "company": r["company"],
        "date": d,  # YYYY-MM-DD
        "open":  float(r["open"])  if pd.notnull(r["open"])  else None,
        "high":  float(r["high"])  if pd.notnull(r["high"])  else None,
        "low":   float(r["low"])   if pd.notnull(r["low"])   else None,
        "close": float(r["close"]) if pd.notnull(r["close"]) else None,

        "change":     float(r["change"])     if pd.notnull(r["change"])     else None,
        "pct_change": float(r["pct_change"]) if pd.notnull(r["pct_change"]) else None,

        "proba_buy": float(r["proba_buy"]) if pd.notnull(r["proba_buy"]) else None,
        "decision": r["decision"],

        "updatedAt": now_iso,
    }

    batch.set(ref, payload, merge=True)
    ops += 1; written += 1

    if ops >= BATCH_LIMIT:
        batch.commit()
        batch = db.batch()
        ops = 0

if ops:
    batch.commit()

print(f"✅ Wrote {written} docs to '{TARGET_DAILY_COL}' (one per symbol+date).")

✅ Loaded model: out\close_model_notrades.joblib
Companies found: 20
Pulled rows: 24960


C:\Users\asma5\AppData\Local\Temp\ipykernel_22268\3213217291.py:138: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["decision"].fillna("لا تشتري", inplace=True)
C:\Users\asma5\AppData\Local\Temp\ipykernel_22268\3213217291.py:139: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For 

✅ Wrote 24960 docs to 'market_predictions_daily' (one per symbol+date).


In [19]:
# --- Diagnostics: check the pulled dataframe and dates BEFORE writing ---

print("Total rows pulled from Firestore:", len(df))
print("Unique symbols:", df["symbol"].nunique())

# how many unique raw dates per symbol from Firestore input:
per_symbol = df.groupby("symbol")[COL_DATE].agg(["min","max","nunique"]).reset_index()
print("Sample per-symbol date stats:")
print(per_symbol.head(10))

print("\nTotal unique source dates in df:", df[COL_DATE].nunique())
print("Min/Max source date:", df[COL_DATE].min(), "->", df[COL_DATE].max())

# After building pred_date:
tmp = df.sort_values(["symbol", COL_DATE]).copy()
tmp["pred_date"] = tmp.groupby("symbol")[COL_DATE].shift(-1)
# keep last-day inheritance the same way the writer does:
mask_last = tmp["pred_date"].isna()
tmp.loc[mask_last, "pred_date"] = tmp.loc[mask_last, COL_DATE]

print("\nUnique pred_date count:", tmp["pred_date"].nunique())
print("Min/Max pred_date:", tmp["pred_date"].min(), "->", tmp["pred_date"].max())

print("\nFirst 5 (symbol, source date -> pred_date) rows:")
print(tmp[["symbol", COL_DATE, "pred_date"]].head())

Total rows pulled from Firestore: 24960
Unique symbols: 20
Sample per-symbol date stats:
  symbol        min        max  nunique
0   1140 2019-01-01 2023-12-31     1248
1   1212 2019-01-01 2023-12-31     1248
2   1810 2019-01-01 2023-12-31     1248
3   2010 2019-01-01 2023-12-31     1248
4   2070 2019-01-01 2023-12-31     1248
5   2080 2019-01-01 2023-12-31     1248
6   2280 2019-01-01 2023-12-31     1248
7   2340 2019-01-01 2023-12-31     1248
8   4004 2019-01-01 2023-12-31     1248
9   4013 2019-01-01 2023-12-31     1248

Total unique source dates in df: 1248
Min/Max source date: 2019-01-01 00:00:00 -> 2023-12-31 00:00:00

Unique pred_date count: 1247
Min/Max pred_date: 2019-01-02 00:00:00 -> 2023-12-31 00:00:00

First 5 (symbol, source date -> pred_date) rows:
     symbol    التاريخ  pred_date
7488   1140 2019-01-01 2019-01-02
7489   1140 2019-01-02 2019-01-03
7490   1140 2019-01-03 2019-01-06
7491   1140 2019-01-06 2019-01-07
7492   1140 2019-01-07 2019-01-08


In [20]:
# list top-level collections
for col in db.collections():
    print("Found collection:", col.id)

Found collection: Favorites
Found collection: Learing
Found collection: Portfolio
Found collection: companies
Found collection: market_predictions
Found collection: market_predictions_daily
Found collection: users


In [22]:
# ===================== Cell 7 — write predictions under companies/*/market_predictions_daily =====================
import pandas as pd, numpy as np, datetime as dt, re
from pathlib import Path
import joblib

import firebase_admin
from firebase_admin import credentials, firestore

# ---------- CONFIG ----------
SERVICE_ACCOUNT_JSON = r"nomu-47a92-firebase-adminsdk-fbsvc-1b2e28026c.json"       # <-- change this
MODEL_PATH           = Path("out/close_model_notrades.joblib")    # or out/change_model_notrades.joblib

COMPANIES_COLLECTION = "companies"
PRICE_SUBCOLLECTION  = "PriceRecords_full"
PRED_SUBCOLLECTION   = "market_predictions_daily"

# Arabic column names used in training (NO trades model = 8 features)
COL_SYMBOL   = "رمز الشركة"
COL_NAME     = "اسم الشركة"
COL_DATE     = "التاريخ"
COL_OPEN     = "إفتتاح"
COL_HIGH     = "الأعلى"
COL_LOW      = "الأدنى"
COL_CLOSE    = "إقفال"
COL_CHANGE   = "التغيير"
COL_PCT      = "% التغيير"
COL_VOLUME   = "الكمية المتداولة"
COL_TURNOVER = "اجمالي القيمة المتداولة (ر.س)"

FEATURES = [COL_OPEN, COL_HIGH, COL_LOW, COL_CLOSE, COL_CHANGE, COL_PCT, COL_VOLUME, COL_TURNOVER]

# Map Firestore raw fields -> our model columns
FS_TO_MODEL = {
    "open":        COL_OPEN,
    "high":        COL_HIGH,
    "low":         COL_LOW,
    "close":       COL_CLOSE,
    "change":      COL_CHANGE,
    "change_pct":  COL_PCT,
    "volume":      COL_VOLUME,
    "value":       COL_TURNOVER,  # turnover SAR
    "date":        COL_DATE,
}

def _norm_key(s: str) -> str:
    s = str(s)
    s = s.replace("إ","ا").replace("أ","ا").replace("آ","ا").replace("ٱ","ا").replace("ى","ي").replace("ة","ه")
    return re.sub(r"\s+", "", s).lower()

def pick(d: dict, *cands, default=None):
    """Pick first existing key among candidates, with Arabic normalization fallback."""
    if not d: return default
    for k in cands:
        if k in d and d[k] is not None:
            return d[k]
    nd = { _norm_key(k): v for k, v in d.items() }
    for k in cands:
        nk = _norm_key(k)
        if nk in nd and nd[nk] is not None:
            return nd[nk]
    return default

# ---------- Firestore + model ----------
if not firebase_admin._apps:
    cred = credentials.Certificate(SERVICE_ACCOUNT_JSON)
    firebase_admin.initialize_app(cred)
db = firestore.client()

clf = joblib.load(MODEL_PATH)
print("✅ Loaded model:", MODEL_PATH)

# ---------- Pull all companies and their price rows ----------
rows = []
companies = list(db.collection(COMPANIES_COLLECTION).stream())
print("Companies found:", len(companies))

for comp in companies:
    comp_id = comp.id
    c = comp.to_dict() or {}
    symbol = str(pick(c, "id", "symbol", COL_SYMBOL, default=comp_id)).strip()
    name   = pick(c, "name", COL_NAME, default=comp_id)

    price_ref = db.collection(COMPANIES_COLLECTION).document(comp_id).collection(PRICE_SUBCOLLECTION)
    for rec in price_ref.stream():
        d = rec.to_dict() or {}
        row = {"company_id": comp_id, "symbol": symbol, "company": name}
        for fs_key, model_key in FS_TO_MODEL.items():
            row[model_key] = pick(d, fs_key, model_key)
        rows.append(row)

df = pd.DataFrame(rows)
if df.empty:
    raise RuntimeError("No data found under companies/*/PriceRecords_full")
print("Pulled rows:", len(df))

# ---------- Types & cleaning ----------
# If your stored dates are D/M/Y strings, add dayfirst=True
df[COL_DATE] = pd.to_datetime(df[COL_DATE], errors="coerce")  # , dayfirst=True

for c in FEATURES:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

df = df.dropna(subset=[COL_DATE, COL_CLOSE]).copy()
df["symbol"] = df["symbol"].astype(str).str.strip()

# Ensure all features exist
fill_defaults = {
    COL_OPEN: np.nan, COL_HIGH: np.nan, COL_LOW: np.nan, COL_CLOSE: np.nan,
    COL_CHANGE: 0.0, COL_PCT: 0.0, COL_VOLUME: 0.0, COL_TURNOVER: 0.0
}
for f in FEATURES:
    if f not in df.columns:
        df[f] = fill_defaults.get(f, 0.0)

# ---------- Predict for ALL days ----------
df = df.sort_values(["symbol", COL_DATE]).copy()
fill_map = {c: fill_defaults.get(c, 0.0) for c in FEATURES}
X = df[FEATURES].fillna(fill_map).astype(np.float64).values

proba = clf.predict_proba(X)
buy_col = np.where(clf.classes_ == 1)[0][0]
df["proba_buy"] = proba[:, buy_col]
df["decision"]  = np.where(df["proba_buy"] >= 0.5, "اشتري", "لا تشتري")

# ---------- Next-day alignment, keep last day ----------
df["pred_date"] = df.groupby("symbol")[COL_DATE].shift(-1)
mask_last = df["pred_date"].isna()
df.loc[mask_last, "pred_date"] = df.loc[mask_last, COL_DATE]  # inherit tail
df["decision"]  = df.groupby("symbol")["decision"].ffill()
df["proba_buy"] = df.groupby("symbol")["proba_buy"].ffill()
df["decision"].fillna("لا تشتري", inplace=True)
df["proba_buy"].fillna(0.0, inplace=True)

# ---------- Build outgoing rows (next-day) ----------
to_write = df[
    ["company_id","symbol","company","pred_date",
     COL_OPEN, COL_HIGH, COL_LOW, COL_CLOSE, COL_CHANGE, COL_PCT,
     "proba_buy","decision"]
].copy()

to_write = to_write.rename(columns={
    "pred_date": "date",
    COL_OPEN: "open", COL_HIGH: "high", COL_LOW: "low",
    COL_CLOSE:"close", COL_CHANGE:"change", COL_PCT:"pct_change"
})

# ---------- PAD FIRST DAY using NEXT day's decision ----------
# 1) first price row per symbol (day1 fields)
first_prices = (
    df.sort_values(["symbol", COL_DATE])
      .groupby("symbol", as_index=False)
      .first()[["company_id","symbol","company", COL_DATE,
                COL_OPEN, COL_HIGH, COL_LOW, COL_CLOSE, COL_CHANGE, COL_PCT]]
      .rename(columns={COL_DATE: "date",
                       COL_OPEN:"open", COL_HIGH:"high", COL_LOW:"low",
                       COL_CLOSE:"close", COL_CHANGE:"change", COL_PCT:"pct_change"})
)

# 2) earliest available prediction per symbol (this is for day2)
earliest_pred = (
    to_write.sort_values(["symbol","date"])
            .groupby("symbol", as_index=False)
            .first()[["symbol","proba_buy","decision"]]
)

# 3) join → first-day rows with next-day decision
first_pad = first_prices.merge(earliest_pred, on="symbol", how="left")
first_pad = first_pad[[
    "company_id","symbol","company","date",
    "open","high","low","close","change","pct_change",
    "proba_buy","decision"
]]

# 4) append & drop any accidental duplicates on (symbol,date)
to_write = pd.concat([to_write, first_pad], ignore_index=True)
to_write = (to_write.sort_values(["symbol","date"])
                    .drop_duplicates(subset=["symbol","date"], keep="first"))

print("Rows to write:", len(to_write))

# ---------- WRITE to companies/{companyId}/market_predictions_daily/{YYYY-MM-DD} ----------
now_iso = dt.datetime.utcnow().isoformat() + "Z"
BATCH_LIMIT = 500
written = 0

for comp_id, g in to_write.groupby("company_id"):
    batch = db.batch()
    ops = 0

    company_doc = db.collection(COMPANIES_COLLECTION).document(comp_id)
    pred_col = company_doc.collection(PRED_SUBCOLLECTION)

    for _, r in g.iterrows():
        date_str = pd.to_datetime(r["date"]).date().isoformat()
        doc_ref = pred_col.document(date_str)

        payload = {
            "symbol": str(r["symbol"]).strip(),
            "company": r["company"],
            "date": date_str,               # YYYY-MM-DD (the day the decision applies to)
            "open":  float(r["open"])  if pd.notnull(r["open"])  else None,
            "high":  float(r["high"])  if pd.notnull(r["high"])  else None,
            "low":   float(r["low"])   if pd.notnull(r["low"])   else None,
            "close": float(r["close"]) if pd.notnull(r["close"]) else None,
            "change":     float(r["change"])     if pd.notnull(r["change"])     else None,
            "pct_change": float(r["pct_change"]) if pd.notnull(r["pct_change"]) else None,
            "proba_buy": float(r["proba_buy"]) if pd.notnull(r["proba_buy"]) else None,
            "decision": r["decision"],
            "updatedAt": now_iso,
        }

        batch.set(doc_ref, payload, merge=True)
        ops += 1
        written += 1

        if ops >= BATCH_LIMIT:
            batch.commit()
            batch = db.batch()
            ops = 0

    if ops:
        batch.commit()

print(f"✅ Wrote {written} docs into subcollections '{PRED_SUBCOLLECTION}' under each company.")

✅ Loaded model: out\close_model_notrades.joblib
Companies found: 20
Pulled rows: 24960


C:\Users\asma5\AppData\Local\Temp\ipykernel_22268\1552275963.py:132: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["decision"].fillna("لا تشتري", inplace=True)
C:\Users\asma5\AppData\Local\Temp\ipykernel_22268\1552275963.py:133: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For 

Rows to write: 24960
✅ Wrote 24960 docs into subcollections 'market_predictions_daily' under each company.


In [23]:
# ===================== Cell 8 — Parity checker (PriceRecords_full vs market_predictions_daily) =====================
import pandas as pd
import numpy as np
import datetime as dt
from typing import Any

import firebase_admin
from firebase_admin import credentials, firestore

# --- config ---
SERVICE_ACCOUNT_JSON = r"nomu-47a92-firebase-adminsdk-fbsvc-1b2e28026c.json"  # <-- edit if needed
COMPANIES_COLLECTION = "companies"
PRICE_SUBCOLLECTION  = "PriceRecords_full"
PRED_SUBCOLLECTION   = "market_predictions_daily"

def to_date_only(v: Any) -> dt.date | None:
    """Normalize Firestore values to date (YYYY-MM-DD). Accepts str, Timestamp, datetime, etc."""
    if v is None:
        return None
    # Firestore Timestamp has .to_datetime()
    if hasattr(v, "to_datetime"):
        try:
            return v.to_datetime().date()
        except Exception:
            pass
    # python datetime / date
    if isinstance(v, dt.datetime):
        return v.date()
    if isinstance(v, dt.date):
        return v
    # string -> parse
    try:
        return pd.to_datetime(str(v), errors="coerce", dayfirst=True).date()
    except Exception:
        return None

# init firestore
if not firebase_admin._apps:
    cred = credentials.Certificate(SERVICE_ACCOUNT_JSON)
    firebase_admin.initialize_app(cred)
db = firestore.client()

companies = list(db.collection(COMPANIES_COLLECTION).stream())
print("Companies:", len(companies))

rows = []
total_only_in_price = 0
total_only_in_pred  = 0

for comp in companies:
    comp_id = comp.id
    data = comp.to_dict() or {}
    name  = data.get("name", comp_id)
    symbol = str(data.get("symbol", data.get("id", comp_id)))

    # --- collect dates from PriceRecords_full (source of truth) ---
    A = set()
    price_ref = db.collection(COMPANIES_COLLECTION).document(comp_id).collection(PRICE_SUBCOLLECTION)
    for doc in price_ref.stream():
        d = doc.to_dict() or {}
        date_val = d.get("date", None)
        dd = to_date_only(date_val)
        if dd: A.add(dd)

    # --- collect dates from market_predictions_daily ---
    B = set()
    pred_ref = db.collection(COMPANIES_COLLECTION).document(comp_id).collection(PRED_SUBCOLLECTION)
    for doc in pred_ref.stream():
        d = doc.to_dict() or {}
        date_val = d.get("date", None)
        dd = to_date_only(date_val)
        # fallback: if field missing, try doc id
        if dd is None:
            dd = to_date_only(doc.id)
        if dd: B.add(dd)

    only_in_price = sorted(A - B)
    only_in_pred  = sorted(B - A)

    total_only_in_price += len(only_in_price)
    total_only_in_pred  += len(only_in_pred)

    rows.append({
        "company_doc_id": comp_id,
        "symbol": symbol,
        "name": name,
        "price_dates": len(A),
        "pred_dates": len(B),
        "missing_in_predictions": len(only_in_price),
        "extra_in_predictions": len(only_in_pred),
        "missing_examples": ", ".join(map(str, only_in_price[:5])),
        "extra_examples": ", ".join(map(str, only_in_pred[:5])),
    })

    # per-company console summary
    print(
        f"[{symbol}] {name} → price={len(A)} / pred={len(B)} | "
        f"missing={len(only_in_price)} extra={len(only_in_pred)}"
        + (f" | missing ex: {', '.join(map(str, only_in_price[:3]))}" if only_in_price else "")
        + (f" | extra ex: {', '.join(map(str, only_in_pred[:3]))}" if only_in_pred else "")
    )

report = pd.DataFrame(rows).sort_values(["missing_in_predictions","extra_in_predictions","symbol"], ascending=[False, False, True]).reset_index(drop=True)
display(report)

csv_path = "parity_report.csv"
report.to_csv(csv_path, index=False, encoding="utf-8-sig")
print("\n✅ Saved detailed report to:", csv_path)

if total_only_in_price == 0 and total_only_in_pred == 0:
    print("✅ Parity check PASSED: prediction dates match price dates for all companies.")
else:
    print(f"⚠️ Parity differences found: missing_total={total_only_in_price}, extra_total={total_only_in_pred}")
    print("   → See 'parity_report.csv' for details.")

Companies: 20


C:\Users\asma5\AppData\Local\Temp\ipykernel_22268\1126431747.py:33: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(str(v), errors="coerce", dayfirst=True).date()


[4013] الشركة السعودية للخدمات الأرضية → price=1248 / pred=1248 | missing=0 extra=0
[2010] الشركة السعودية للصناعات الأساسية → price=1248 / pred=1248 | missing=0 extra=0
[2070] الشركة السعودية للصناعات الدوائية والمستلزمات الطبية → price=1248 / pred=1248 | missing=0 extra=0
[8190] الشركة المتحدة للتأمين التعاوني → price=1248 / pred=1248 | missing=0 extra=0
[4030] الشركة الوطنية السعودية للنقل البحري → price=1248 / pred=1248 | missing=0 extra=0
[4210] المجموعة السعودية للأبحاث والإعلام → price=1248 / pred=1248 | missing=0 extra=0
[1140] بنك البلاد → price=1248 / pred=1248 | missing=0 extra=0
[2340] شركة ارتيكس للاستثمار الصناعي → price=1248 / pred=1248 | missing=0 extra=0
[7010] شركة الإتصالات السعودية → price=1248 / pred=1248 | missing=0 extra=0
[2080] شركة الغاز والتصنيع الأهلية → price=1248 / pred=1248 | missing=0 extra=0
[2280] شركة المراعي → price=1248 / pred=1248 | missing=0 extra=0
[4170] شركة المشروعات السياحية → price=1248 / pred=1248 | missing=0 extra=0
[4160] شركة ثمار التنمي

,company_doc_id,symbol,name,price_dates,pred_dates,missing_in_predictions,extra_in_predictions,missing_examples,extra_examples
0,بنك البلاد,1140,بنك البلاد,1248,1248,0,0,,
1,مجموعة استرا الصناعية,1212,مجموعة استرا الصناعية,1248,1248,0,0,,
2,مجموعة سيرا القابضة,1810,مجموعة سيرا القابضة,1248,1248,0,0,,
3,الشركة السعودية للصناعات الاساسية,2010,الشركة السعودية للصناعات الأساسية,1248,1248,0,0,,
4,الشركة السعودية للصناعات الدوائية والمستلزمات ...,2070,الشركة السعودية للصناعات الدوائية والمستلزمات ...,1248,1248,0,0,,
5,شركة الغاز والتصنيع الاهلية,2080,شركة الغاز والتصنيع الأهلية,1248,1248,0,0,,
6,شركة المراعي,2280,شركة المراعي,1248,1248,0,0,,
7,شركة ارتيكس للاستثمار الصناعي,2340,شركة ارتيكس للاستثمار الصناعي,1248,1248,0,0,,
8,شركة دله للخدمات الصحية,4004,شركة دله للخدمات الصحية,1248,1248,0,0,,
9,الشركة السعودية للخدمات الارضية,4013,الشركة السعودية للخدمات الأرضية,1248,1248,0,0,,



✅ Saved detailed report to: parity_report.csv
✅ Parity check PASSED: prediction dates match price dates for all companies.
